#***I am scraping the Zillow website to get additional information for the IOWA State Housing Dataset for School Rank using beautifulsoup4***

In [ ]:
#data importing
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

#load data
df_base_dataset = pd.read_csv('/content/gdrive/MyDrive/Dataset_Realestate.csv')
print(df_base_dataset.shape)
df_base_dataset.head(1)
print(df_base_dataset.info()) #data information like numbers of features and data type




Mounted at /content/gdrive
(5890, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5890 entries, 0 to 5889
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             5890 non-null   int64  
 1   property_id      5890 non-null   int64  
 2   address          5890 non-null   object 
 3   latitude         5774 non-null   float64
 4   longitude        5774 non-null   float64
 5   price            5873 non-null   float64
 6   currency         5890 non-null   object 
 7   bathrooms        5342 non-null   float64
 8   bedrooms         5426 non-null   float64
 9   area             5290 non-null   object 
 10  land_area        443 non-null    object 
 11  zestimate        3968 non-null   float64
 12  rent_zestimate   4535 non-null   float64
 13  days_on_zillow   5886 non-null   float64
 14  sold_date        0 non-null      float64
 15  is_zillow_owned  5890 non-null   bool   
 16  image            5890 

In [ ]:
#filter IOWA state data
def get_state_data(base_data_set, state_code):
    return base_data_set[base_data_set['address'].str.contains(state_code.upper())]

#this is the dataframe for IA state going forward
df = get_state_data(df_base_dataset, 'IA')

#print info
df.head(2)
print(df.shape)
print(df.info())

(757, 22)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 14 to 5878
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             757 non-null    int64  
 1   property_id      757 non-null    int64  
 2   address          757 non-null    object 
 3   latitude         730 non-null    float64
 4   longitude        730 non-null    float64
 5   price            746 non-null    float64
 6   currency         757 non-null    object 
 7   bathrooms        663 non-null    float64
 8   bedrooms         713 non-null    float64
 9   area             670 non-null    object 
 10  land_area        90 non-null     object 
 11  zestimate        443 non-null    float64
 12  rent_zestimate   484 non-null    float64
 13  days_on_zillow   756 non-null    float64
 14  sold_date        0 non-null      float64
 15  is_zillow_owned  757 non-null    bool   
 16  image            757 non-null    object 
 17  list

#Scrape data to find school rating in IOWA state and merge on property link and saved in csv.

In [ ]:
#for each entry, scrape homeunion date to find total returns
#info-value propertyDetailTotalReturn
import time
from io import BytesIO
import requests
from bs4 import BeautifulSoup
import seaborn as sns
from sklearn import tree
from googlesearch import search

header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}

def find_school_rating(link):
      err_res = ['0','0','0']
      err = False
      try:
        r = requests.get(link, headers=header)
      except Exception as err:
        return err_res, err
      #print(r.content)
      soup = BeautifulSoup(r.content, 'html5lib')
      #print(soup.prettify())
      d = soup.find(id="ds-nearby-schools-list")
      rating = []
      try:
        for item in d.select('li'):
            try:
              r = ((item.find_all("div")[0]).find_all("div")[0]).find_all('span')[0]
            except Exception as err:
              rating.append('0')
              err = True

            if r is None:
              rating.append('0')
            else:
              rating.append(r.text)
      except:
        return err_res, err

      return rating, err

test = 'https://www.zillow.com/homedetails/109-Sunrise-Fls-Cibolo-TX-78108/70983496_zpid/'

school_ratings = {}
def scrape_school_rating():
  for i in df['property_url']:
    #print(i)
    time.sleep(1)
    res, err = find_school_rating(i)
    if err:
      print('Err: retrying')
      res, err = find_school_rating(i)
      print(i)
      print(res)
    school_ratings[i] = res


scrape_school_rating()

for i in school_ratings:
    print(i)



#search_home('https://www.zillow.com/homedetails/109-Sunrise-Fls-Cibolo-TX-78108/70983496_zpid/')



https://www.zillow.com/homedetails/2863-Spring-Rose-Cir-Coralville-IA-52241/247261885_zpid/
https://www.zillow.com/homedetails/215-3rd-Ave-SW-Waverly-IA-50677/76679888_zpid/
https://www.zillow.com/homedetails/813-Cherry-St-NW-Bondurant-IA-50035/238337348_zpid/
https://www.zillow.com/homedetails/2405-36th-St-Des-Moines-IA-50310/803919_zpid/
https://www.zillow.com/homedetails/1418-Dorene-Blvd-Carter-Lake-IA-51510/76832920_zpid/
https://www.zillow.com/homedetails/1527-Mamie-Eisenhower-Ave-Boone-IA-50036/86879884_zpid/
https://www.zillow.com/homedetails/1310-25th-Ave-N-Fort-Dodge-IA-50501/76895419_zpid/
https://www.zillow.com/homedetails/702-W-Park-Rd-Iowa-City-IA-52246/76766140_zpid/
https://www.zillow.com/homedetails/519-Main-St-Muscatine-IA-52761/87031597_zpid/
https://www.zillow.com/homedetails/200-Jordan-Dr-West-Des-Moines-IA-50265/859850_zpid/
https://www.zillow.com/homedetails/120-S-Cardinal-Dr-APT-10-Clarinda-IA-51632/2067947053_zpid/
https://www.zillow.com/homedetails/2084-Wedgewo

In [ ]:
print(len(school_ratings))

import csv

# open file for writing, "w" is writing
w = csv.writer(open("output.csv", "w"))
for key, val in school_ratings.items():
    # write every key and value to file
    w.writerow([key, val])

641
